In [ ]:
import keras
import numpy as np
from LSTM import LSTMScratch
import pandas as pd
from Embedding import Embedding

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import Callable, Tuple
from keras.src.backend import Variable
import keras
import tensorflow as tf

class LSTMScratch():
    def __init__(
            self,
            units: int, # neuron
            keras_weight: list[Variable],   # weight from keras
            activation: Callable[[NDArray[np.float64]], NDArray[np.float64]] = np.tanh, # activation function
    ):
        self.units = units
        self.kernel, self.recurrent_kernel, self.bias = keras_weight
        self.h_t = []
        self.c_t = []
        self.activation = activation

    def fit(self, input_feature: NDArray[np.float64]):
        self.input_feature = input_feature

    def predict(self, input_feature):
        if isinstance(input_feature, tf.Tensor):
            input_feature = input_feature.numpy() # type: ignore
        sample = input_feature.shape[0]
        timestep = input_feature.shape[1]
        for i in range(sample):
            current_sample = input_feature[i]
            self.temp_h = [np.zeros(self.units)]
            self.temp_c = [np.zeros(self.units)]

            for j in range(timestep):
                h_t, c_t = self.calc(current_sample[j]) 
                self.temp_h.append(h_t)
                self.temp_c.append(c_t)

            self.h_t.append(self.temp_h[-1])
            self.c_t.append(self.temp_c[-1])
        return self.h_t

    def calc(self, x_t: NDArray[np.float64]):
        w_dot_x = x_t @ self.kernel
        u_dot_h = self.temp_h[-1] @ self.recurrent_kernel
        result = w_dot_x + u_dot_h + self.bias 
        k_i, k_f, k_c, k_o = np.split(result, 4, axis=0)

        i_t, f_t, o_t = self.sigmoid(k_i), self.sigmoid(k_f), self.sigmoid(k_o)

        c_tilde = self.activation(k_c)
        c_t = f_t * self.temp_c[-1] + i_t * c_tilde
        
        h_t = o_t * self.activation(c_t)

        return h_t, c_t     

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))


In [14]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(5))

In [15]:
X = np.random.rand(100, 10, 2)
y = np.random.rand(100, 1)

In [16]:
model.compile(optimizer="adam", loss="mse")
model.fit(X, y, epochs=1, batch_size=16)
pred = model.predict(X[:5])
print(pred)

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.3871
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
[[-0.42746967  0.27468234 -0.2032304   0.18286541  0.19074002]
 [-0.367071    0.20233795 -0.17620282  0.21743988  0.16649815]
 [-0.4097815   0.26991102 -0.21765153  0.16863544  0.17366153]
 [-0.3943891   0.25758183 -0.21340916  0.17390485  0.16845536]
 [-0.4342258   0.24703404 -0.18287928  0.2976126   0.18360014]]


In [17]:
weight = model.get_weights()

In [18]:
x = LSTMScratch(5, weight)

In [19]:
x.predict(X[:5])

[array([-0.4274697 ,  0.27468238, -0.20323043,  0.18286545,  0.19074007]),
 array([-0.36707103,  0.20233806, -0.17620289,  0.21743987,  0.16649819]),
 array([-0.40978149,  0.26991107, -0.21765158,  0.16863546,  0.17366151]),
 array([-0.39438912,  0.25758183, -0.2134092 ,  0.17390485,  0.16845541]),
 array([-0.43422584,  0.24703413, -0.18287929,  0.2976126 ,  0.18360017])]

In [20]:
import tensorflow as tf

df = pd.read_csv("../dataset/rnn/test.csv")
x = df["text"]
y_text = df["label"]

vectorization = keras.layers.TextVectorization(
    max_tokens=1000, output_mode="int", output_sequence_length=5
)
vectorization.adapt(x)

x_vectorized = vectorization(tf.constant(x))  # keluaran tensor int

# Embedding
embed = Embedding(
    input_dim=len(vectorization.get_vocabulary()), output_dim=100
)

embedded_vectors = embed(x_vectorized)

lstm_layer = keras.layers.LSTM(5)
lstm_output = lstm_layer(embedded_vectors) 

print(lstm_output.numpy()[:5])

[[ 0.03532427  0.00679311 -0.00832508 -0.02876124  0.00749967]
 [ 0.02399009  0.00930851  0.00026791  0.00793475 -0.00633904]
 [ 0.00851871  0.0011163  -0.0164678   0.01314454  0.00840074]
 [ 0.01997273  0.03011882  0.00491694 -0.02831706 -0.00176409]
 [ 0.01101295 -0.01723707 -0.00916221  0.01752237  0.01089727]]


In [21]:
scratch_lstm = LSTMScratch(5, lstm_layer.get_weights())
pred = scratch_lstm.predict(embedded_vectors)
pred[:5]

[array([ 0.03532428,  0.00679312, -0.00832509, -0.02876125,  0.00749968]),
 array([ 0.02399009,  0.00930851,  0.00026791,  0.00793476, -0.00633904]),
 array([ 0.00851871,  0.0011163 , -0.01646781,  0.01314455,  0.00840075]),
 array([ 0.01997274,  0.03011883,  0.00491693, -0.02831707, -0.00176409]),
 array([ 0.01101295, -0.01723708, -0.00916222,  0.01752238,  0.01089727])]

In [22]:
# import pandas as pd
# from tensorflow import keras

# # Load data
# df = pd.read_csv("../dataset/rnn/test.csv")
# x = df["text"]
# y_text = df["label"]

# # Text vectorization layer
# vectorization = keras.layers.TextVectorization(
#     max_tokens=1000, output_mode="int", output_sequence_length=5
# )
# vectorization.adapt(x)

# # Define model input
# inputs = keras.Input(shape=(1,), dtype="string")  # sesuai dengan input x (string)
# x_vectorized = vectorization(inputs)              # gunakan vectorization sebagai layer
# embed = keras.layers.Embedding(
#     input_dim=len(vectorization.get_vocabulary()), output_dim=100
# )(x_vectorized)
# lstm = keras.layers.LSTM(5)(embed)
# outputs = keras.layers.Dense(1, activation="sigmoid")(lstm)

# # Build model
# model = keras.Model(inputs, outputs)
# model.summary()